In [1]:
import pandas as pd
import dash 
from dash import html,dcc
from dash.dependencies import Input,Output,State
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update
import datetime as dt

In [2]:
df=pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/Historical_Wildfires.csv')
df.head(10)

,Region,Date,Estimated_fire_area,Mean_estimated_fire_brightness,Mean_estimated_fire_radiative_power,Mean_confidence,Std_confidence,Var_confidence,Count,Replaced
0,NSW,1/4/2005,8.680000,312.266667,42.400000,78.666667,2.886751,8.333333,3,R
1,NSW,1/5/2005,16.611250,322.475000,62.362500,85.500000,8.088793,65.428571,8,R
2,NSW,1/6/2005,5.520000,325.266667,38.400000,78.333333,3.214550,10.333333,3,R
3,NSW,1/7/2005,6.264000,313.870000,33.800000,92.200000,7.529940,56.700000,5,R
4,NSW,1/8/2005,5.400000,337.383333,122.533333,91.000000,7.937254,63.000000,3,R
5,NSW,1/9/2005,13.721429,314.757143,64.542857,83.142857,6.175990,38.142857,7,R
6,NSW,1/10/2005,16.625455,324.800000,58.554545,91.454545,6.088738,37.072727,11,R
7,NSW,1/11/2005,32.784375,318.446875,66.812500,91.000000,10.132456,102.666667,16,R
8,NSW,1/12/2005,16.486154,323.238462,40.607692,87.769231,8.623165,74.358974,13,R
9,NSW,1/13/2005,42.640000,325.376667,87.573333,86.666667,7.499206,56.238095,15,R


In [5]:
df['Month']=pd.to_datetime(df['Date']).dt.month_name()
df['Year']=pd.to_datetime(df['Date']).dt.year

In [13]:
app=dash.Dash(__name__)
app.layout=html.Div(children=[
    html.H1('Australia Wildfire Dashboard',
            style={'textAlign':'center',
                   'color':'#503D36',
                   'font-size':26}),
    html.Div([html.H2('Select Region:',style={'margin-right':'2em'}),
              dcc.RadioItems(['NSW','QL','SA','TA','VI','WA'],'NSW',id='region',inline=True)
              ]),
    html.Div([
       html.H2('Select Year:',style={'margin-right':'2em'}),
       dcc.Dropdown(df.Year.unique(),value=2005,id='year')
    ]),

    html.Div([
       html.Div([],id='plot1'),
       html.Div([],id='plot2')
    ],style={'display':'flex'}),
    html.Br(),
    html.Br(),
    html.Div()
])

@app.callback(Output(component_id='plot1',component_property='children'),
              Output(component_id='plot2',component_property='children'),
              Input(component_id='region',component_property='value'),
              Input(component_id='year',component_property='value'))

def reg_year_display(input_region,input_year):
   region_data=df[df['Region']==input_region]
   y_r_data=region_data[region_data['Year']==input_year]

   est_data=y_r_data.groupby('Month')['Estimated_fire_area'].mean().reset_index()
   fig1 = px.pie(est_data, values='Estimated_fire_area', names='Month', title="{} : Monthly Average Estimated Fire Area in year {}".format(input_region,input_year))
   #Plot two - Monthly Average Count of Pixels for Presumed Vegetation Fires
   veg_data = y_r_data.groupby('Month')['Count'].mean().reset_index()
   fig2 = px.bar(veg_data, x='Month', y='Count', title='{} : Average Count of Pixels for Presumed Vegetation Fires in year {}'.format(input_region,input_year))
    
   return [dcc.Graph(figure=fig1),
            dcc.Graph(figure=fig2) ]
if(__name__=='__main__'):
   app.run_server()